In [ ]:
!pip install yahoofinancials
%pip install yahoofinancials

In [ ]:
from data_visualization import graphs
from data_visualization import load_net_worth_history, get_depot_history

# Load data

In [ ]:
net_worth_history, avg_return = load_net_worth_history("~/Desktop/net_worth_history.csv")
net_worth_history.head()

In [ ]:
proposition_history, quote_history, value_history = get_depot_history()

# Fortune history line plot

Using just a line diagram.

In [ ]:
graphs.get_fortune_history_line_plot(net_worth_history, avg_return).show()

# Fortune proposition history

Using a stacked area chart.

In [ ]:
graphs.get_fortune_history_area_plot(net_worth_history).show()

# Current fortune proposition (pie)

In [ ]:
graphs.get_current_fortune_proposition_pie(net_worth_history).show()

# Depot value fluctuation histogram

In [ ]:
graphs.get_depot_value_fluctuation_histogram(net_worth_history).show()

# Net worth bubble chart

In [ ]:
graphs.get_net_worth_bubble_chart(net_worth_history).show()

# Depot proposition

In [ ]:
graphs.get_depot_proposition_by_shares_pie(proposition_history).show()

In [ ]:
graphs.get_depot_proposition_by_value_pie(value_history).show()
value_history.tail()

In [ ]:
graphs.get_quotes_history_line_plot(quote_history).show()

In [ ]:
graphs.get_depot_proposition_history_line_plot(proposition_history).show()

In [ ]:
graphs.get_depot_value_history_line_plot(value_history).show()

In [ ]:
graphs.get_depot_value_history_area_plot(value_history).show()